In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv,eig, det
from math import sqrt,exp,pi,log
import pickle
from joblib import dump, load
from sklearn.preprocessing import StandardScaler

import shap

from joblib import dump,load
from sklearn import utils,preprocessing
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

import matplotlib as mpl
import matplotlib.pyplot as plt

KeyboardInterrupt: 

# read data

In [ ]:
data = pd.read_excel('list-MAZ-025.xlsx',header=0)
data = data.iloc[:,1:]
X = data.iloc[:,1:]
Y = data.iloc[:,0]
data

# Shapley additive explanations

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(best_regr)
shap_values = explainer(X)
shap_values

In [ ]:
f1 = plt.figure()
shap.plots.beeswarm(shap_values)
f1.savefig('Shap_beeswarm.pdf', bbox_inches = 'tight')
plt.clf()

In [ ]:
f2 = plt.figure()
shap.plots.bar(shap_values)
f2.savefig('Shap_bar.pdf', bbox_inches = 'tight')
plt.clf()

# mutual information

In [ ]:
def p_mkde(x,X,h):    
    N = X.shape[0]
    Sxy = np.cov(X.transpose())
    sum = 0
    if Sxy.shape == ():
        d = 1
        detS=Sxy
        for i in range(N):
            p2 = np.dot(np.dot((np.array(x)-np.array(X[i])),1/Sxy),(np.array(x)-np.array(X[i])).transpose())
            sum = sum+1/sqrt(pow((2*pi),d)*detS)*exp(-p2/(2*pow(h,2)))
    else:
        d = X.shape[1]
        detS=det(Sxy)
        for i in range(N):
            p2 = np.dot(np.dot((np.array(x.transpose())-np.array(X.iloc[i,:])),np.linalg.inv(Sxy)),(np.array(x.transpose())-np.array(X.iloc[i,:])).transpose())
            sum = sum+1/sqrt(pow((2*pi),d)*detS)*exp(-p2/(2*pow(h,2)))
    pxy = 1/(N*pow(h,d))*sum
    return pxy

In [ ]:
def MutualInfo(X,Y):
    d = 2
    nx = len(X)
    hx = pow(4/(d+2),1/(d+2))*pow(nx,-1/(d+4))
    Xall = pd.concat([X,Y], axis=1)
    sum1 = 0
    for i in range(nx):
        pxy=p_mkde(pd.DataFrame([X[i],Y[i]]),Xall,hx)
        px=p_mkde(X[i],X,hx)
        py=p_mkde(Y[i],Y,hx)
        sum1=sum1+log(pxy/(px*py))
    Ixy=sum1/nx
    lambda1 = sqrt(1-exp(-2*Ixy))
    return Ixy,lambda1

In [ ]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
Num = X.shape[1]
Mi_list = []

In [ ]:
for i in range(Num):
    Ixy,lambda1 = MutualInfo(X.iloc[:,i],Y)
    Mi_list.append(Ixy)

In [ ]:
Mi_list = pd.DataFrame(Mi_list).transpose()
Mi_list.columns = data.columns.values.tolist()[1:]
Mi_list

# MI values based on Shap bar

In [ ]:
[Mi_list.iloc[0,3],Mi_list.iloc[0,4],Mi_list.iloc[0,54],Mi_list.iloc[0,32],Mi_list.iloc[0,36],Mi_list.iloc[0,35],Mi_list.iloc[0,33],Mi_list.iloc[0,19],Mi_list.iloc[0,41]]

# Pearson correlation coefficients

In [ ]:
PCC_GH = abs(data.corr().iloc[0:1,1:])
PCC_GH

# PCC values based on Shap bar

In [ ]:
[PCC_GH.iloc[0,3],PCC_GH.iloc[0,4],PCC_GH.iloc[0,54],PCC_GH.iloc[0,32],PCC_GH.iloc[0,36],PCC_GH.iloc[0,35],PCC_GH.iloc[0,33],PCC_GH.iloc[0,19],PCC_GH.iloc[0,41]]

# Spearman correlation coefficient

In [ ]:
SCC_GH = abs(data.corr('spearman').iloc[0:1,1:])
SCC_GH

# SCC values based on Shap bar

In [ ]:
[SCC_GH.iloc[0,3],SCC_GH.iloc[0,4],SCC_GH.iloc[0,54],SCC_GH.iloc[0,32],SCC_GH.iloc[0,36],SCC_GH.iloc[0,35],SCC_GH.iloc[0,33],SCC_GH.iloc[0,19],SCC_GH.iloc[0,41]]